计算给定数据集的香农熵

In [6]:
from math import log

def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    # 为所有可能的分类 创建字典
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        # 以2为底求对数
        shannonEnt -= prob*log(prob,2)
    return shannonEnt

In [8]:
def createDataSet():
    dataSet = [
        [1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no'],
    ]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [9]:
myDat, labels = createDataSet()

In [10]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [11]:
calcShannonEnt(myDat)

0.9709505944546686

熵越高，则混合的数据也越多，我们可以在数据集中添加更多的分类，观察熵是如何变化的。
这里我们增加第三个名为maybe的分类，测试熵的变化：

In [12]:
myDat[0][-1] = 'maybe'

In [13]:
myDat

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [14]:
calcShannonEnt(myDat)

1.3709505944546687

按照给定特征划分数据集

In [16]:
def splitDataSet(dataSet, axis, value):
    """输入：待划分的数据集、划分数据集的特征、需要返回的特征的值"""
    # 满足分类特征对应value的子集
    retDataSet = []
    for featVec in dataSet:
        # 每个样本的对应属性如果有指定的值，则将该属性剔除，保留其他属性，生成该特征对应值的【子集】
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
        
    return retDataSet

In [17]:
a = [1, 2, 3]
b = [4, 5, 6]

In [18]:
a.append(b)

In [19]:
a

[1, 2, 3, [4, 5, 6]]

In [20]:
a = [1, 2, 3]
a.extend(b)

In [21]:
a

[1, 2, 3, 4, 5, 6]

In [22]:
myDat, labels = createDataSet()

In [23]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [24]:
splitDataSet(myDat, 0, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [25]:
splitDataSet(myDat, 0, 0)

[[1, 'no'], [1, 'no']]

选择最好的数据集划分方式

In [31]:
def choosebestfeaturetosplit(dataset):   
    """就算出信息增益之后选取信息增益值最高的特征作为下一次分类的标准"""
    numfeatures=len(dataset[0])-1     #计算特征数量，列表【0】表示列的数量，-1是减去最后的类别特征label
    baseentropy=calcShannonEnt(dataset)   #计算数据集的信息熵
    bestinfogain=0.0;bestfeature=-1
    
    # 遍历特征的index
    for i in range(numfeatures):  
        # 求该特征一共有多少个取值
        featlist=[example[i] for example in dataset]
        uniquevals=set(featlist)   #确定某一特征下所有可能的取值
        newentropy=0.0
        
        # 遍历每一个对应特征的取值
        for value in uniquevals:
            subdataset=splitDataSet(dataset,i,value)#抽取在该特征的每个取值下其他特征的值组成新的子数据集
            prob=len(subdataset)/float(len(dataset))#计算该特征下的每一个取值对应的概率（或者说所占的比重）
            #计算该特征下每一个取值的子数据集的信息熵，即特征a中，a1占a1...an所有值的占比*信息熵，再求和
            newentropy +=prob*calcShannonEnt(subdataset)
            
        #计算每个特征的信息增益
        infogain=baseentropy-newentropy
        #  print("第%d个特征是的取值是%s，对应的信息增益值是%f"%((i+1),uniquevals,infogain))
        if(infogain>bestinfogain):
            bestinfogain=infogain
            bestfeature=i
    # print("第%d个特征的信息增益最大，所以选择它作为划分的依据，其特征的取值为%s,对应的信息增益值是%f"%((i+1),uniquevals,infogain))
    return bestfeature

In [27]:
myDat, labels = createDataSet()

In [32]:
choosebestfeaturetosplit(myDat)

0

In [ ]:
import operator
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys()